### Data preprocessing
In this tutorial, we show the data preprocessing steps for firefighting cost prediction with tree-based models.

### 1. Import libraries
For the preprocessing, we will use pandas for loading the dataset and performing some operations with its rows and columns. We also use datetime to deal with date formatting and other operations. The specific operations we performed will be explained as we advance in the tutorial.

In [2]:
import pandas as pd
from datetime import datetime

### 2. Load the dataset 
We load the dataset from data/lfb_incident.csv. This file was not added to this repo because it is very large. However, you can download it from Kaggle via the links provided on README on the root of the repo. 

We then select the columns we intend to use. The choice of columns was at first based on what we thought would work best ofr predicting fires in a real world scenario. We asked ourselves: right after the fire department receives a fire call and sends the firemen to check it, what kind of data they have about the incident?
- Date and time
- Place (e.g. they know if it is in a building, a house, a park, etc.)
- Number of fire engines sent to work on the incident

Therefore, we selected the columns that contain these data, as well as the variable we want to predict: cost. We later check for correlation between these variables in another notebook (preprocessing/2_check_correlation.ipynb).

At this point we did not do any filtering, so the dataset contains 1,465,060 rows.

In [3]:
used_cols = ['DateOfCall', 'CalYear', 'HourOfCall', 'IncidentGroup', 
'PropertyType', 'PumpHoursRoundUp', 'NumPumpsAttending', 
'Notional Cost (£)', 'PropertyCategory']
ds = pd.read_csv('data/lfb_incident.csv', usecols=used_cols)
print("Count:", len(ds))

Count: 1465060


### 2. Drop NaN values
We drop any rows containing NaN values. Current row count: 1,453,312.

In [4]:
ds = ds.dropna()
print("Count:", len(ds))

Count: 1453312


### 3. Duplicate DateOfCall column
We duplicate the column DateOfCall and convert it to the format month/day/year. This will be needed later for joining with weather data. See step 9 for more info.

In [5]:
ds2 = pd.DataFrame(ds)
ds2['Date'] = ds2['DateOfCall']
ds2['Date'] = pd.to_datetime(ds2['Date']).dt.strftime('%m/%d/%Y')

### 4. Convert DateOfCall column
We convert the column DateOfCall to month only. This is necessary because we need this feature to be discrete (have distinct, countable values). However, there are too many combination of day/month/year. Therefore, we decided to reduce that to 12 categories, the 12 months of the year. For our model, monthly precision is good enough.

In [6]:
ds2['DateOfCall'] = pd.to_datetime(ds2['DateOfCall']).dt.month
ds2[0:5]

,DateOfCall,CalYear,HourOfCall,IncidentGroup,PropertyCategory,PropertyType,NumPumpsAttending,PumpHoursRoundUp,Notional Cost (£),Date
0,1,2009,0,Special Service,Road Vehicle,Car,2.0,1.0,255.0,01/01/2009
2,1,2009,0,Fire,Outdoor,Road surface/pavement,1.0,1.0,255.0,01/01/2009
3,1,2009,0,Fire,Outdoor,Domestic garden (vegetation not equipment),1.0,1.0,255.0,01/01/2009
4,1,2009,0,Fire,Outdoor,Cycle path/public footpath/bridleway,2.0,1.0,255.0,01/01/2009
5,1,2009,0,False Alarm,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,2.0,1.0,255.0,01/01/2009


### 5. Standardizing numerical variables
We converted all numerical variables (e.g. number of pumps, cost, etc.) to int64.

In [7]:
ds2['NumPumpsAttending'] = ds2['NumPumpsAttending'].astype('int64')
ds2['Notional Cost (£)'] = ds2['Notional Cost (£)'].astype('int64')
ds2['PumpHoursRoundUp'] = ds2['PumpHoursRoundUp'].astype('int64')
ds2['HourOfCall'] = ds2['HourOfCall'].astype('int64')

### 6. Standardizing categorical variables
Categorical variables (e.g. incident group, property type, etc.) were originally strings (e.g. "building" or "house" for PropertyType). We factorized them or, in other words, we converted them to categorical integer values (e.g. "building" becomes 1, "house" becomes 2, etc.).

In [8]:
ds2['IncidentGroup'] = pd.factorize(ds2['IncidentGroup'])[0]
ds2['PropertyType'] = pd.factorize(ds2['PropertyType'])[0]
ds2['PropertyCategory'] = pd.factorize(ds2['PropertyCategory'])[0]

### 7. Saving
We made a lot of changes to the dataset so far so at this point we save to disk. Next, we will join this dataset with another one, containing weather data. Current row count: 1,453,312.

In [9]:
ds2.to_csv('data/london_clean.csv', index=False)
len(ds2)

1453312

### 8. Load and prepare weather data
We load weather data from data/london_weather.csv. We convert the Date column to the format month/day/year, so it matches the format in our other dataset. This is very important because this column will be used for the join operation we will do next.

In [10]:
weather_data = pd.read_csv('data/london_weather.csv')
weather_data['date'] = pd.to_datetime(weather_data['date'], format='%Y%m%d').dt.strftime('%m/%d/%Y')
weather_data = weather_data.rename(columns={'date': 'Date'})
weather_data[0:5]

,Date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,01/01/1979,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,01/02/1979,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,01/03/1979,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,01/04/1979,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,01/05/1979,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0


### 9. Join the datasets
While this operation is frequently called "join" in the context of relational databases, for example, in pandas it is called "merge". The principle, however, is the same: we choose a column that exists is both datasets and we perform an "inner join" (intersection of keys from both datasets).

In [11]:
ds_merged = pd.merge(ds2, weather_data, on='Date')
ds_merged[0:5]

,DateOfCall,CalYear,HourOfCall,IncidentGroup,PropertyCategory,PropertyType,NumPumpsAttending,PumpHoursRoundUp,Notional Cost (£),Date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,1,2009,0,0,0,0,2,1,255,01/01/2009,8.0,0.0,13.0,3.5,1.5,-0.5,0.0,103010.0,0.0
1,1,2009,0,1,1,1,1,1,255,01/01/2009,8.0,0.0,13.0,3.5,1.5,-0.5,0.0,103010.0,0.0
2,1,2009,0,1,1,2,1,1,255,01/01/2009,8.0,0.0,13.0,3.5,1.5,-0.5,0.0,103010.0,0.0
3,1,2009,0,1,1,3,2,1,255,01/01/2009,8.0,0.0,13.0,3.5,1.5,-0.5,0.0,103010.0,0.0
4,1,2009,0,2,2,4,2,1,255,01/01/2009,8.0,0.0,13.0,3.5,1.5,-0.5,0.0,103010.0,0.0


### 10. Remove rows without temparature data
Temperature is an important component in fire incidents (e.g. hot temperatures make it hard to put of a fire, or make some kinds of fires more likely). Therefore, we need it for our predictions and all rows that do not have this data can be removed. Current row count: 1,286,617.

In [12]:
ds_merged = ds_merged[ds_merged['mean_temp'].notna()]
print(len(ds_merged))

1286617


### 11. Creating cost categories
Our output feature is the fire pumps' notional cost in pound sterling (£). The cost value was originally a continuous numerical variable, but we converted it to a categorical variable, dividing and categorizing the numerical value in intervals of £300. For example, all records of cost between £0.00 and £300.00 fall under category 1, all records of cost between £300.01 and £500.00 fall under category 2, and so on. All records with costs larger than £1100.00 fall under category 5. 

IMPORTANT: this step MUST be performed after the join and all other filtering steps. If we do this first and filter after, we risk not having all categories from 1 to 5 represented in the dataset, which will cause problems when training the tree models.


In [13]:
# check to which category each cost belongs to
cost_cat = []
for item in ds_merged['Notional Cost (£)'].values:
    if item < 300:
        cost_cat.append(0)
    elif item >= 300 and item < 500:
        cost_cat.append(1)
    elif item >= 500 and item < 700:
        cost_cat.append(2)
    elif item >= 700 and item < 900:
        cost_cat.append(3)
    elif item >= 900 and item < 1100:
        cost_cat.append(4)    
    else:
        cost_cat.append(5)

# add cost categories to dataset
ds_merged['CostCat'] = cost_cat

# check wheter the categories exist and how many corresponding rows they have
print(ds_merged['CostCat'].value_counts())
print(len(ds_merged))

0    701094
1    425134
2    104288
5     24706
3     16375
4     15020
Name: CostCat, dtype: int64
1286617


### 12. Save
Save the final result to disk. This will be used to train or tree models.

In [14]:
ds_merged.to_csv('data/london_clean.csv', index=False)